In [2]:
"""
raytrace.py
----------------

A very simple example of using scene cameras to generate
rays for image reasons.

Install `pyembree` for a speedup (600k+ rays per second)
"""

import numpy as np
import PIL.Image

import trimesh


In [3]:

# test on a sphere mesh
mesh = trimesh.primitives.Sphere()

mesh = trimesh.load("../models/bunny.ply")

# Normalize the mesh using trimesh functionality
mesh.apply_translation(-mesh.centroid)
scale_factor = 1.0 / mesh.extents.max()
mesh.apply_scale(scale_factor)




# create some rays
ray_origins = np.array([[0, 0, -5], [2, 2, -10]])
ray_directions = np.array([[0, 0, 1], [0, 0, 1]])

# scene will have automatically generated camera and lights
scene = mesh.scene()

# any of the automatically generated values can be overridden
# set resolution, in pixels
scene.camera.resolution = [640, 480]
# set field of view, in degrees
# make it relative to resolution so pixels per degree is same
scene.camera.fov = [
    60,
    60,
]  # * (scene.camera.resolution / scene.camera.resolution.max())


# convert the camera to rays with one ray per pixel
origin, vectors, pixels = scene.camera_rays()
# if len(origin) > 10_000:
#     print(len(origin))
#     origin = ray_origins

# intersects_location requires origins to be the same shape as vectors
# origins = np.tile(np.expand_dims(origin, 0), (len(vectors), 1))

# run the mesh- ray test
points, index_ray, index_tri = mesh.ray.intersects_location(
    ray_origins=origin, ray_directions=vectors, multiple_hits=False
)

# you could also do this against the single camera Z vector
depth = trimesh.util.diagonal_dot(points - origin[index_ray], vectors[index_ray])
# find pixel locations of actual hits
pixel_ray = pixels[index_ray]

# create a numpy array we can turn into an image
# doing it with uint8 creates an `L` mode greyscale image
a = np.zeros(scene.camera.resolution, dtype=np.uint8)

# scale depth against range (0.0 - 1.0)
depth_float = (depth - depth.min()) / depth.ptp()

# convert depth into 0 - 255 uint8
depth_int = (depth_float * 255).astype(np.uint8)
# assign depth to correct pixel locations
a[pixel_ray[:, 0], pixel_ray[:, 1]] = depth_int

# create a PIL image from the depth queries
img = PIL.Image.fromarray(a)

img.show()

# # stack rays into line segments for visualization as Path3D
# ray_visualize = trimesh.load_path(
#     np.hstack((origin, origin + vectors)).reshape(-1, 2, 3)
# )

# # make mesh transparent- ish
# mesh.visual.face_colors = [100, 100, 100, 100]

# # create a visualization scene with rays, hits, and mesh
# scene = trimesh.Scene([mesh, ray_visualize, trimesh.points.PointCloud(locations)])

# # display the scene
# scene.show()


In [24]:
%%timeit
mask = np.random.randint(0, 640*480, int(640*480/2))

1.75 ms ± 4.15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [27]:
%%timeit
m = np.random.choice(640*480, int(640*480/2), replace=False)

5.44 ms ± 29.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit
mask = np.random.rand(640*480) < 0.5
mask = np.random.rand(640*480) < 0.5

4.26 ms ± 59.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
len(vectors)

307200

In [47]:
%%timeit
mask = np.random.rand(640*480) < 0.5
points, index_ray, index_tri = mesh.ray.intersects_location(
    ray_origins=origin[mask], ray_directions=vectors[mask], multiple_hits=False
)

71.2 ms ± 715 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [48]:
%%timeit
points, index_ray, index_tri = mesh.ray.intersects_location(
    ray_origins=origin, ray_directions=vectors, multiple_hits=False
)

# # you could also do this against the single camera Z vector
# depth = trimesh.util.diagonal_dot(points - origin[index_ray], vectors[index_ray])
# # find pixel locations of actual hits
# pixel_ray = pixels[index_ray]

112 ms ± 3.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:

rotation_angles = [15, 70, -20] # arbitrary angles
translation_vector = [0.5, .2, 3] # arbitrary translation


# Convert rotation angles from degrees to radians for trimesh
rotation_radians = np.radians(rotation_angles)
rotation_matrix = trimesh.transformations.euler_matrix(
    rotation_radians[0], rotation_radians[1], rotation_radians[2]
)

# Combine rotation and translation into a single transformation matrix
transformation_matrix = rotation_matrix.copy()
transformation_matrix[:3, 3] = translation_vector

# Apply the transformation to the bunny mesh
scene = mesh.scene()
mesh
# transformed = mesh.copy()
# transformed = transformed.apply_transform(transformation_matrix)

scene.show()

In [57]:
%%timeit
mesh.bounding_box_oriented.projected(normal=[0, 0, 1])

1.23 ms ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
